### $Train-Validation-Test$

### $example$

```python
# 训练集
train_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST('datasets/mnist_data',
                train=True,
                download=True,
                transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),                   
                torchvision.transforms.Normalize((0.1307, ), (0.3081, )) 
    ])), batch_size=batch_size,shuffle=True)

# 验证集
test_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST('datasets/mnist_data/',
                train=False,
                download=True,
                transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307, ), (0.3081, ))
    ])),batch_size=batch_size,shuffle=False)
```

$test\space while\space train$
```python
for epoch in range(epochs):
    for batch_idx,(data,target) in enumerate(train_loader):
        ...
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        logits = forward(data)
        test_loss += criteon(logits, target).item()

        pred = logits.data.max(1)[1]
        correct += pred.eq(target.data).sum()
        ...

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
```

$train-val-test$
```python
print('train:',len(train_db),'test:',len(test_db))
train_db,val_db = torch.utils.data.random_split(train_db.[50000,10000])
print('db1:',len(train_db),'db2:',len(test_db))
train_loader = torch.utils.data.DataLoader(
    train_db,
    batch_size = batch_size,
    shuffle = True)
val_loader = torch.utils.data.DataLoader(
    val_db,
    batch_size = batch_sze,
    shuffle = True)
```

### $K-fold\space cross-validation$

In [1]:
import torch
import torch.nn as nn
import torchvision 
from torchvision import datasets,transforms
from torch.nn import functional as F
import torch.optim as optim

In [2]:
batch_size = 200
learning_rate  = 1e-2
epochs = 10

In [3]:
train_db =  datasets.MNIST('datasets/mnist_data',
                train=True,
                download=True,
                transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),                       # 数据类型转化
                torchvision.transforms.Normalize((0.1307, ), (0.3081, )) # 数据归一化处理
    ]))

train_loader = torch.utils.data.DataLoader(
        train_db,
        batch_size = batch_size,
        shuffle = True)

test_db = datasets.MNIST('datasets/mnist_data/',
                train=False,
                download=True,
                transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307, ), (0.3081, ))
    ]))

test_loader = torch.utils.data.DataLoader(
        test_db,
        batch_size = batch_size,
        shuffle = True
)

In [4]:
print('train:', len(train_db), 'test:', len(test_db))
train_db, val_db = torch.utils.data.random_split(train_db, [50000, 10000])
print('db1:', len(train_db), 'db2:', len(val_db))
train_loader = torch.utils.data.DataLoader(
    train_db,
    batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(
    val_db,
    batch_size=batch_size, shuffle=True)

train: 60000 test: 10000
db1: 50000 db2: 10000


In [5]:
class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 10),
            nn.LeakyReLU(inplace=True),
        )

    def forward(self, x):
        x = self.model(x)

        return x

In [6]:
device = torch.device('cuda:0')
net = MLP().to(device)
optimizer = optim.SGD(net.parameters(), lr=learning_rate)
criteon = nn.CrossEntropyLoss().to(device)

for epoch in range(epochs):

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28)
        data, target = data.to(device), target.cuda()

        logits = net(data)
        loss = criteon(logits, target)

        optimizer.zero_grad()
        loss.backward()
        # print(w1.grad.norm(), w2.grad.norm())
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


    test_loss = 0
    correct = 0
    for data, target in val_loader:
        data = data.view(-1, 28 * 28)
        data, target = data.to(device), target.cuda()
        logits = net(data)
        test_loss += criteon(logits, target).item()

        pred = logits.data.max(1)[1]
        correct += pred.eq(target.data).sum()

    test_loss /= len(val_loader.dataset)
    print('\nVAL set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))



test_loss = 0
correct = 0
for data, target in test_loader:
    data = data.view(-1, 28 * 28)
    data, target = data.to(device), target.cuda()
    logits = net(data)
    test_loss += criteon(logits, target).item()

    pred = logits.data.max(1)[1]
    correct += pred.eq(target.data).sum()

test_loss /= len(test_loader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

Train Epoch: 0 [0/50000 (0%)]	Loss: 2.301248
Train Epoch: 0 [20000/50000 (40%)]	Loss: 1.991530
Train Epoch: 0 [40000/50000 (80%)]	Loss: 1.640657

VAL set: Average loss: 0.0075, Accuracy: 5424/10000 (54%)

Train Epoch: 1 [0/50000 (0%)]	Loss: 1.430360
Train Epoch: 1 [20000/50000 (40%)]	Loss: 1.216766
Train Epoch: 1 [40000/50000 (80%)]	Loss: 1.114616

VAL set: Average loss: 0.0058, Accuracy: 6429/10000 (64%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 1.145380
Train Epoch: 2 [20000/50000 (40%)]	Loss: 0.934966
Train Epoch: 2 [40000/50000 (80%)]	Loss: 0.814469

VAL set: Average loss: 0.0037, Accuracy: 7966/10000 (80%)

Train Epoch: 3 [0/50000 (0%)]	Loss: 0.677089
Train Epoch: 3 [20000/50000 (40%)]	Loss: 0.599673
Train Epoch: 3 [40000/50000 (80%)]	Loss: 0.727854

VAL set: Average loss: 0.0026, Accuracy: 8602/10000 (86%)

Train Epoch: 4 [0/50000 (0%)]	Loss: 0.555265
Train Epoch: 4 [20000/50000 (40%)]	Loss: 0.310324
Train Epoch: 4 [40000/50000 (80%)]	Loss: 0.400627

VAL set: Average loss: 0.0019, Acc